Instalacje

In [13]:
#!pip install tensorflow

In [19]:
#!pip install tensorboard

In [17]:
#!pip install torch

In [400]:
#!git clone https://github.com/ultralytics/yolov5

In [405]:
#!pip install -r requirements.txt

Trenowanie modelu detekcji kart

In [1]:
%cd yolov5

C:\Users\rafal\yolov5


In [9]:
# trenowanie danych (przygotowane w LabelStudio) - model wykonuje detekcję kart(klasa cart) ze zdjęcia
!python train.py --img 640 --batch 16 --epochs 30 --data custom_data.yaml --weights yolov5s.pt --cache ram --workers 2
# najlepszy model dokładność 0.957 jest w runs\train\exp7
# w zasadzie jednokrotne wykonanie

Error decoding JSON from C:\Users\rafal\AppData\Roaming\Ultralytics\settings.json. Starting with an empty dictionary.
Creating new Ultralytics Settings v0.0.6 file âś… 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\rafal\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


train: weights=yolov5s.pt, cfg=, data=custom_data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github:  YOLOv5 is out of date by 5 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5  v7.0-378-g2f74455a Python-3.12.4 torch-2.5.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1,

In [59]:
# Przygotowanie danych do uczenia perceptrona tj. zdjęcia w skali szarości przeskalowane do konkretnego wymiaru
from PIL import Image
from numpy import asarray
import os
from sklearn.preprocessing import LabelEncoder

image_data = []
image_labels = []
files = os.listdir('C:\\Users\\rafal\\perceptron_trainset_to_model_grey_size\\')
for f in files:
    img = Image.open('C:\\Users\\rafal\\perceptron_trainset_to_model_grey_size\\{}'.format(f))    
    img_array = np.array(img)
    image_data.append(img_array)
    label = f.replace(".jpg.png","")[-1] # w przygotowaniu danych trochę nieelegancko zrobiłem nazwę, ale ten kod wybiera z nazwy literę
    image_labels.append(label)
  
image_data = np.array(image_data)
image_data_flattened = image_data.reshape(image_data.shape[0], -1)
image_labels = np.array(image_labels)

# Tablica etykiet
labels = image_labels

# Utworzenie enkodera i dopasowanie do danych
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

import pickle

# zapis LabelEncoder
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [363]:
#Utworzenie modelu do rozpoznawania karta > litera i jego zapis
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
import numpy as np

X = image_data_flattened
y = encoded_labels

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3)

perceptron = Perceptron(max_iter=30, eta0=0.05)

perceptron.fit(X_train,y_train)

perceptron.score(X_test,y_test) # ok. 93%

import pickle
# zapisujemy model do odtworzenia
with open('perceptron_model.pkl', 'wb') as file:
    pickle.dump(perceptron, file)

In [3]:
# odtworzenie modelu
import pickle
with open('C:\\Users\\rafal\\perceptron_model.pkl', 'rb') as file:
    perceptron = pickle.load(file)

with open('C:\\Users\\rafal\\label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

Model, który ze zdjęcia identyfikuje karty, bada jakie zawierają litery i proponuje najbardziej punktowane słowa

In [5]:
#tutaj kod który bierze obrazek i używając najlepszego modelu detekcji rozpoznaje karty i umieszcza je jako pojedyncze zdjęcia
!python detect.py --weights runs/train/exp7/weights/best.pt --source C:\Users\rafal\7words_to_do\example.jpg --img 416 --save-csv --save-crop

detect: weights=['runs/train/exp7/weights/best.pt'], source=C:\Users\rafal\7words_to_do\example.jpg, data=data\coco128.yaml, imgsz=[416, 416], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=True, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-378-g2f74455a Python-3.12.4 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 C:\Users\rafal\7words_to_do\example.jpg: 320x416 8 carts, 40.2ms
Speed: 0.0ms pre-process, 40.2ms inference, 6.0ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs\detect\exp52


In [7]:
exp_no = 'exp52'

In [9]:
# rozpoznane karty z YOLO przerabiam do formatu użytecznego dla neuronu
from PIL import Image
import os

files = os.listdir("C:\\Users\\rafal\\yolov5\\runs\\detect\\{}\\crops\\cart\\".format(exp_no))
os.mkdir("C:\\Users\\rafal\\yolov5\\runs\\detect\\{}\\crops\\cart_to_model".format(exp_no))
for f in files:
    img = Image.open("C:\\Users\\rafal\\yolov5\\runs\\detect\\{}\\crops\\cart\\{}".format(exp_no,f)).convert('L').resize((480,650))
    img.save("C:\\Users\\rafal\\yolov5\\runs\\detect\\{}\\crops\\cart_to_model\\{}.png".format(exp_no,f))

In [11]:
# zdjęcia liter do rozpoznania zapisuje w postaci arraya
from PIL import Image
from numpy import asarray
import os
import numpy as np
image_data = []

files = os.listdir("C:\\Users\\rafal\\yolov5\\runs\\detect\\{}\\crops\\cart_to_model\\".format(exp_no))
for f in files:
    img = Image.open('C:\\Users\\rafal\\yolov5\\runs\\detect\\{}\\crops\\cart_to_model\\{}'.format(exp_no,f))    
    img_array = np.array(img)
    image_data.append(img_array)
    
image_data = np.array(image_data)
image_data_flattened = image_data.reshape(image_data.shape[0], -1)
X = image_data_flattened

# używam modelu, żeby ze zdjęcia uzyskać literę
perceptron.predict(X)
# encoduje labelki na litery
lista_liter = label_encoder.inverse_transform(perceptron.predict(X))
lista_liter = list(lista_liter)

In [13]:
lista_liter

['w', 'r', 'c', 'r', 'k', 'z', 'k', 'p']

In [15]:
#defincija funkcji, która sprawdza ile unikalnych liter ze zdjęcia jest w słownikowym słowie
def ile_liter_z_listy_w_slowie(row, lista_liter):
    row_ = list(row['aa'])
    letter_points = 0
    
    for letter in lista_liter:
        try:
            row_.remove(letter)
            letter_points = letter_points + 1
        except:
            continue

    return letter_points

In [17]:
#załadowanie słownika oraz dodanie kolumny z punktacją
import pandas as pd
slowa = pd.read_csv('C:\\Users\\rafal\\slowa.txt')
slowa['points'] = slowa.apply(ile_liter_z_listy_w_slowie,axis=1,args=(lista_liter,))

In [18]:
#przykładowe słowa z największą punktacją
slowa.sort_values(by="points", ascending= False)

,aa,points
2184854,przykrywkowcze,8
2184166,przykrawaczki,8
2107149,przeciwnakrętkę,8
2107150,przeciwnakrętki,8
2107151,przeciwnakrętko,8
...,...,...
1694567,ołysienie,0
521297,inset,0
1694568,ołysieniem,0
1694569,ołysieniom,0
